# Evaluación del modelo de lenguaje

In [3]:
import argparse
import glob
import json
import re
import html
import math
import random
import pickle
import gc
from pathlib import Path
from lxml import etree
from collections import Counter
from typing import Dict, List, Tuple
import nltk

Se definen los PATHS a los datos. En caso de necesitarlo aquí es donde se modifican para poder utilizar los datos locales.

In [4]:
PATH_TO_BAC = "data/BAC/blogs"  # Cambia esta ruta por la de tu carpeta con archivos XML
OUTPUT_BAC_JSONL = "data/processed/BAC.jsonl"
PATH_TO_20NG = "data/20news-18828/"  # Cambia esta ruta por la de tu carpeta con archivos de 20 Newsgroups
OUTPUT_20NG_JSONL = "data/processed/20news.jsonl"

OUTPUT_20NG_TOKENIZED_JSONL = "data/processed/20news_tokenized.jsonl"
OUTPUT_BAC_TOKENIZED_JSONL = "data/processed/BAC_tokenized.jsonl"


GROUP_ID = "0100"
OUTPUT_20NG_SPLITS_JSONL = f"splits/20N_{GROUP_ID}_training.jsonl"
OUTPUT_BAC_SPLITS_JSONL = f"splits/BAC_{GROUP_ID}_training.jsonl"


TEST_20NG_SPLITS_JSONL = f"splits/20N_{GROUP_ID}_testing.jsonl"
TEST_BAC_SPLITS_JSONL = f"splits/BAC_{GROUP_ID}_testing.jsonl"

## Carga de los archivos Pickle

In [5]:
with open('ngrams/unigrams_20n.pkl', 'rb') as f:
    unigrams_20n = pickle.load(f)

with open('ngrams/context_counts_unigrams_20n.pkl', 'rb') as f:
    context_counts_uni_20n = pickle.load(f)

with open('ngrams/bigrams_20n.pkl', 'rb') as f:
    bigrams_20n = pickle.load(f)

with open('ngrams/context_counts_bigrams_20n.pkl', 'rb') as f:
    context_counts_bi_20n = pickle.load(f)

with open('ngrams/trigrams_20n.pkl', 'rb') as f:
    trigrams_20n = pickle.load(f)

with open('ngrams/context_counts_trigrams_20n.pkl', 'rb') as f:
    context_counts_tri_20n = pickle.load(f)


In [6]:
print("=== INFORMACIÓN GENERAL ===")
print(f"Tipo unigrams: {type(unigrams_20n)}")
print(f"Tipo bigrams: {type(bigrams_20n)}")
print(f"Tipo trigrams: {type(trigrams_20n)}")

print(f"\nCantidad de unigrams: {len(unigrams_20n)}")
print(f"Cantidad de bigrams: {len(bigrams_20n)}")
print(f"Cantidad de trigrams: {len(trigrams_20n)}")

=== INFORMACIÓN GENERAL ===
Tipo unigrams: <class 'dict'>
Tipo bigrams: <class 'dict'>
Tipo trigrams: <class 'dict'>

Cantidad de unigrams: 72174
Cantidad de bigrams: 941694
Cantidad de trigrams: 2342730


In [7]:
print("\n=== PRIMEROS 10 UNIGRAMS ===")
if isinstance(unigrams_20n, dict):
    # Si es diccionario, mostrar primeros items
    for i, (key, value) in enumerate(list(unigrams_20n.items())[:10]):
        print(f"{key}: {value}")
else:
    # Si es lista, mostrar primeros elementos
    print(unigrams_20n[:10])


print("\n=== PRIMEROS 10 BIGRAMS ===")
if isinstance(bigrams_20n, dict):
    for i, (key, value) in enumerate(list(bigrams_20n.items())[:10]):
        print(f"{key}: {value}")
else:
    print(bigrams_20n[:10])

print("\n=== PRIMEROS 10 TRIGRAMS ===")
if isinstance(trigrams_20n, dict):
    for i, (key, value) in enumerate(list(trigrams_20n.items())[:10]):
        print(f"{key}: {value}")
else:
    print(trigrams_20n[:10])


=== PRIMEROS 10 UNIGRAMS ===
('<s>',): 0.03759524512819843
('is',): 0.009200650824709796
('it',): 0.007095915046812705
('worth',): 8.600416801983155e-05
('to',): 0.015476670349451091
('run',): 0.00026421394311974816
('atm',): 7.343809413458102e-06
('at',): 0.0022356187812213896
('all',): 0.001921956521384357
('?',): 0.006548393256098218

=== PRIMEROS 10 BIGRAMS ===
('<s>', 'is'): 0.0045646554858498984
('is', 'it'): 0.007413400129131629
('it', 'worth'): 0.00019022256039566293
('worth', 'it'): 0.0006327372764786795
('it', 'to'): 0.007997994016635828
('to', 'run'): 0.002305278789040046
('run', 'atm'): 9.486123156981787e-05
('atm', 'at'): 8.308178016560969e-05
('at', 'all'): 0.009886808272778089
('all', '?'): 0.0008219178082191781

=== PRIMEROS 10 TRIGRAMS ===
('<s>', 'is', 'it'): 0.0034260543274329063
('is', 'it', 'worth'): 0.00020512539999453
('it', 'worth', 'it'): 0.0001246623727404945
('worth', 'it', 'to'): 0.0001938548027527382
('it', 'to', 'run'): 0.00019152370789898493
('to', 'run'

In [17]:
with open('ngrams/unigrams_bac.pkl', 'rb') as f:
    unigrams_bac = pickle.load(f)

with open('ngrams/context_counts_unigrams_bac.pkl', 'rb') as f:
    context_counts_uni = pickle.load(f)

with open('ngrams/bigrams_bac.pkl', 'rb') as f:
    bigrams_bac = pickle.load(f)

with open('ngrams/context_counts_bigrams_bac.pkl', 'rb') as f:
    context_counts_bi = pickle.load(f)

with open('ngrams/trigrams_bac.pkl', 'rb') as f:
    trigrams_bac = pickle.load(f)

with open('ngrams/context_counts_trigrams_bac.pkl', 'rb') as f:
    context_counts_tri = pickle.load(f)

In [9]:
print("=== INFORMACIÓN GENERAL ===")
print(f"Tipo unigrams: {type(unigrams_bac)}")
print(f"Tipo bigrams: {type(bigrams_bac)}")
print(f"Tipo trigrams: {type(trigrams_bac)}")

print(f"\nCantidad de unigrams: {len(unigrams_bac)}")
print(f"Cantidad de bigrams: {len(bigrams_bac)}")
print(f"Cantidad de trigrams: {len(trigrams_bac)}")

=== INFORMACIÓN GENERAL ===
Tipo unigrams: <class 'dict'>
Tipo bigrams: <class 'dict'>
Tipo trigrams: <class 'dict'>

Cantidad de unigrams: 386329
Cantidad de bigrams: 9861802
Cantidad de trigrams: 38191007


In [10]:
print("\n=== PRIMEROS 10 UNIGRAMS ===")
if isinstance(unigrams_bac, dict):
    # Si es diccionario, mostrar primeros items
    for i, (key, value) in enumerate(list(unigrams_bac.items())[:10]):
        print(f"{key}: {value}")
else:
    # Si es lista, mostrar primeros elementos
    print(unigrams_bac[:10])

print("\n=== PRIMEROS 10 BIGRAMS ===")
if isinstance(bigrams_bac, dict):
    for i, (key, value) in enumerate(list(bigrams_bac.items())[:10]):
        print(f"{key}: {value}")
else:
    print(bigrams_bac[:10])

print("\n=== PRIMEROS 10 TRIGRAMS ===")
if isinstance(trigrams_bac, dict):
    for i, (key, value) in enumerate(list(trigrams_bac.items())[:10]):
        print(f"{key}: {value}")
else:
    print(trigrams_bac[:10])


=== PRIMEROS 10 UNIGRAMS ===
('<s>',): 0.049583917069526466
('just',): 0.003068443153250799
('a',): 0.015964224080597066
('few',): 0.0005275043847319924
('weeks',): 0.00019377825625771782
('ago',): 0.00022502227066114588
(',',): 0.03351788743120267
('an',): 0.0017274539492573284
('army',): 3.6994915785941965e-05
('vehicle',): 1.5243017932854284e-05

=== PRIMEROS 10 BIGRAMS ===
('<s>', 'just'): 0.003949886858158352
('just', 'a'): 0.026262081898931946
('a', 'few'): 0.020871799078313522
('few', 'weeks'): 0.010006815452686694
('weeks', 'ago'): 0.007911751498090267
('ago', ','): 0.01893060926181133
(',', 'an'): 0.0013558523786191357
('an', 'army'): 0.0006349036301363389
('army', 'vehicle'): 1.0213252716725222e-05
('vehicle', 'was'): 0.0001492844641202512

=== PRIMEROS 10 TRIGRAMS ===
('<s>', 'just', 'a'): 0.005538221903859929
('just', 'a', 'few'): 0.004268959128759845
('a', 'few', 'weeks'): 0.006228705401451933
('few', 'weeks', 'ago'): 0.0026908605382232648
('weeks', 'ago', ','): 0.0021970

## Cálculo de la perplejidad

In [11]:
def load_test_sentences(file_path: str) -> List[List[str]]:
    """
    Carga y tokeniza las oraciones de un archivo JSONL de prueba.
    """
    test_sentences = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            sentence = data['sentence']

            test_sentences.append(sentence)
    return test_sentences


In [12]:

test_sentences_20n = load_test_sentences(TEST_20NG_SPLITS_JSONL)
test_sentences_bac = load_test_sentences(TEST_BAC_SPLITS_JSONL)

In [13]:
print(f"\nCantidad de oraciones de prueba en 20 Newsgroups: {len(test_sentences_20n)}")
print(f"Cantidad de oraciones de prueba en BAC: {len(test_sentences_bac)}")


Cantidad de oraciones de prueba en 20 Newsgroups: 57592
Cantidad de oraciones de prueba en BAC: 1782586


Se genera una clase que permita generar un modelo de lenguaje a partir de las probabilidades de n-gramas ya calculadas. De esta manera se pueden manejar métodos comunes como el cálculo de la perplejidad y la generación de oraciones.

In [53]:
class NgramModel:
    """
    Modelo de n-gramas que maneja predicción, generación y evaluación.
    """
    
    def __init__(self, ngram_probs: Dict[Tuple[str, ...], float], 
                 context_counts: Dict[Tuple[str, ...], int],
                 n: int, 
                 laplace: float = 1.0):
        """
        Inicializa el modelo con las probabilidades de n-gramas y context_counts reales.
        
        Args:
            ngram_probs: Diccionario con probabilidades de n-gramas entrenados
            context_counts: Diccionario con conteos de contextos del entrenamiento
            n: Tamaño del n-grama (1, 2, 3, etc.)
            laplace: Parámetro de suavizado de Laplace
        """
        self.ngram_probs = ngram_probs
        self.context_counts = context_counts if context_counts else {}
        self.n = n
        self.laplace = laplace
        
        # Calcular información derivada
        self._calculate_vocab_info()
    
    def _calculate_vocab_info(self):
        """Calcula solo vocabulary y vocab_size desde los n-gramas."""
        
        # Extraer vocabulario completo
        self.vocabulary = set()
        for ngram in self.ngram_probs.keys():
            self.vocabulary.update(ngram)
        self.vocab_size = len(self.vocabulary)
        
        # Para unigramas, calcular total si es necesario
        if self.n == 1:
            self.total_unigrams = len(self.ngram_probs)
    
    def predict_next_word(self, context: List[str]) -> Tuple[str, float]:
        """
        Predice la siguiente palabra más probable dado un contexto.
        
        Args:
            context: Lista de palabras que forman el contexto
        
        Returns:
            Tuple[str, float]: (palabra_predicha, probabilidad)
        """
        
        # Para unigramas, muestreo aleatorio ponderado
        if self.n == 1:
            candidates = {ngram[0]: prob for ngram, prob in self.ngram_probs.items()}
            
            if candidates:
                # Muestreo ponderado por probabilidad
                words = list(candidates.keys())
                probs = list(candidates.values())
                chosen_word = random.choices(words, weights=probs, k=1)[0]
                chosen_prob = candidates[chosen_word]
                return chosen_word, chosen_prob
        
        # Para n-gramas superiores, tomar las últimas n-1 palabras
        else:
            if len(context) >= self.n - 1:
                relevant_context = context[-(self.n-1):]
            else:
                relevant_context = context
            
            context_tuple = tuple(relevant_context)
            
            # Buscar todos los n-gramas que empiecen con este contexto
            candidates = {}
            for ngram, prob in self.ngram_probs.items():
                if ngram[:-1] == context_tuple:
                    next_word = ngram[-1]
                    candidates[next_word] = prob
            
            if candidates:
                best_word = max(candidates, key=candidates.get)
                best_prob = candidates[best_word]
                return best_word, best_prob

        uniform_prob = 1.0 / self.vocab_size
        return random.choice(list(self.vocabulary)), uniform_prob
    
    def get_ngram_probability(self, ngram: Tuple[str, ...]) -> float:
        """
        Calcula la probabilidad de un n-grama específico usando suavizado de Laplace.
        
        Args:
            ngram: N-grama a evaluar
            
        Returns:
            float: Probabilidad del n-grama
        """
        
        # Si el n-grama fue visto durante entrenamiento
        if ngram in self.ngram_probs:
            return self.ngram_probs[ngram]
        
        # Si no fue visto, aplicar suavizado de Laplace correcto
        if self.n == 1:
            # Para unigramas no vistos - usar aproximación o 1/vocab_size para OOV
            return 1.0 / self.vocab_size
        else:
            # Para n-gramas superiores
            context = ngram[:-1]
            
            if context in self.context_counts:
                # Contexto conocido: usar count real
                context_count = self.context_counts[context]
                return self.laplace / (context_count + self.laplace * self.vocab_size)
            else:
                # Contexto no visto: probabilidad uniforme
                return 1.0 / self.vocab_size
    
    def calculate_perplexity(self, test_sentences: List[List[str]]) -> float:
        """
        Calcula la perplejidad del modelo sobre un conjunto de prueba.
        
        Args:
            test_sentences: Lista de oraciones tokenizadas para evaluar
        
        Returns:
            float: Valor de perplejidad
        """
        
        total_log_prob = 0.0
        total_ngrams = 0
        unseen_count = 0
        
        for sentence in test_sentences:
            # Generar n-gramas de la oración
            for i in range(len(sentence) - self.n + 1):
                ngram = tuple(sentence[i:i + self.n])
                
                if ngram in self.ngram_probs:
                    # N-grama visto: usar su probabilidad
                    prob = self.ngram_probs[ngram]
                else:
                    # N-grama no visto: usar suavizado correcto
                    unseen_count += 1
                    prob = self.get_ngram_probability(ngram)
                
                total_log_prob += math.log(prob)
                total_ngrams += 1
        
        if total_ngrams == 0:
            return float('inf')
        
        # Calcular perplejidad
        avg_log_prob = total_log_prob / total_ngrams
        perplexity = math.exp(-avg_log_prob)

        print(f"Estadísticas para {self.n}-gramas:")
        print(f"  Total n-gramas evaluados: {total_ngrams}")
        print(f"  N-gramas no vistos: {unseen_count}")
        print(f"  Perplejidad: {perplexity:.4f}")

        return perplexity

    def get_model_info(self) -> Dict:
        """Retorna información del modelo."""
        return {
            'n': self.n,
            'laplace': self.laplace,
            'vocab_size': self.vocab_size,
            'num_ngrams': len(self.ngram_probs),
            'num_contexts': len(self.context_counts)
        }

In [54]:
model_20n_uni = NgramModel(unigrams_20n, context_counts_uni_20n, n=1)
model_20n_bi = NgramModel(bigrams_20n, context_counts_bi_20n, n=2)
model_20n_tri = NgramModel(trigrams_20n, context_counts_tri_20n, n=3)

In [37]:
model_20n_tri.get_model_info()

{'n': 3,
 'laplace': 1.0,
 'vocab_size': 72174,
 'num_ngrams': 2342730,
 'num_contexts': 939236}

In [56]:
model_bac_uni = NgramModel(unigrams_bac, context_counts_uni, n=1)
model_bac_bi = NgramModel(bigrams_bac, context_counts_bi, n=2)
model_bac_tri = NgramModel(trigrams_bac, context_counts_tri, n=3)

In [36]:
model_20n_tri.get_model_info()

{'n': 3,
 'laplace': 1.0,
 'vocab_size': 72174,
 'num_ngrams': 2342730,
 'num_contexts': 939236}

## Evaluación de los modelos

In [38]:
def evaluate_all_ngrams(test_sentences: List[List[str]], 
                       unigram_model: NgramModel, 
                       bigram_model: NgramModel, 
                       trigram_model: NgramModel) -> Dict[str, float]:
    """
    Evalúa perplejidad para todos los modelos de n-gramas usando objetos NgramModel.
    
    Args:
        test_sentences: Oraciones de prueba
        unigram_model: Modelo de unigramas (NgramModel)
        bigram_model: Modelo de bigramas (NgramModel)
        trigram_model: Modelo de trigramas (NgramModel)
    
    Returns:
        Dict con perplejidades de cada modelo
    """
    
    results = {}
    
    print("=" * 50)
    print("EVALUACIÓN DE PERPLEJIDAD")
    print("=" * 50)
    
    # Evaluar unigramas
    print("\n   UNIGRAMAS:")
    results['unigrams'] = unigram_model.calculate_perplexity(test_sentences)
    
    # Evaluar bigramas
    print("\n   BIGRAMAS:")
    results['bigrams'] = bigram_model.calculate_perplexity(test_sentences)
    
    # Evaluar trigramas
    print("\n   TRIGRAMAS:")
    results['trigrams'] = trigram_model.calculate_perplexity(test_sentences)
    
    # Resumen comparativo
    print("\n" + "=" * 50)
    print("RESUMEN COMPARATIVO")
    print("=" * 50)
    
    for model, perplexity in results.items():
        print(f"{model.upper():<12}: {perplexity:>8.2f}")
    
    
    return results

In [49]:
perplexity_results = evaluate_all_ngrams(
    test_sentences_20n,
    model_20n_uni,
    model_20n_bi,
    model_20n_tri
)

EVALUACIÓN DE PERPLEJIDAD

   UNIGRAMAS:
Estadísticas para 1-gramas:
  Total n-gramas evaluados: 1503195
  N-gramas no vistos: 3095
  Perplejidad: 637.8866

   BIGRAMAS:
Estadísticas para 2-gramas:
  Total n-gramas evaluados: 1445603
  N-gramas no vistos: 136116
  Perplejidad: 981.5244

   TRIGRAMAS:
Estadísticas para 3-gramas:
  Total n-gramas evaluados: 1388011
  N-gramas no vistos: 422560
  Perplejidad: 7127.5982

RESUMEN COMPARATIVO
UNIGRAMS    :   637.89
BIGRAMS     :   981.52
TRIGRAMS    :  7127.60


In [40]:
perplexity_results = evaluate_all_ngrams(
        test_sentences_bac,
        model_bac_uni,
        model_bac_bi,
        model_bac_tri
    )

EVALUACIÓN DE PERPLEJIDAD

   UNIGRAMAS:
Estadísticas para 1-gramas:
  Total n-gramas evaluados: 35828187
  N-gramas no vistos: 16252
  Perplejidad: 656.3425

   BIGRAMAS:
Estadísticas para 2-gramas:
  Total n-gramas evaluados: 34045601
  N-gramas no vistos: 1528478
  Perplejidad: 734.1268

   TRIGRAMAS:
Estadísticas para 3-gramas:
  Total n-gramas evaluados: 32263015
  N-gramas no vistos: 7196407
  Perplejidad: 11842.1412

RESUMEN COMPARATIVO
UNIGRAMS    :   656.34
BIGRAMS     :   734.13
TRIGRAMS    : 11842.14


## Predicción de palabras

In [58]:
def generate_sentence(model: NgramModel,
                     initial_context: List[str],
                     stop_tokens: List[str] = ['</s>'],
                     max_length: int = 20,
                     include_context: bool = True) -> List[str]:
    """
    Genera una oración completa usando un modelo de n-gramas entrenado.
    
    Args:
        model: Modelo de n-gramas entrenado (NgramModel)
        initial_context: Contexto inicial para empezar la generación
        stop_tokens: Lista de tokens que indican fin de oración
        max_length: Número máximo de palabras a generar (sin contar contexto inicial)
        include_context: Si True, incluye el contexto inicial en la oración final
    
    Returns:
        List[str]: Oración generada como lista de palabras
    """
    
    generated_words = []
    current_context = initial_context.copy()
    
    for i in range(max_length):
        next_word, prob = model.predict_next_word(current_context)
        
        generated_words.append(next_word)
        
        if next_word in stop_tokens:
            break
        
        if model.n == 1:
            pass  
        else:
            current_context = current_context[1:] + [next_word]
            if len(current_context) > model.n - 1:
                current_context = current_context[-(model.n-1):]
    
    if include_context:
        full_sentence = initial_context + generated_words
    else:
        full_sentence = generated_words
    
    return full_sentence

## Experimentación con mejores modelos

A continuación se intentará generar oraciones con los mejores modelos unigramas, bigramas y trigramas. De esta manera se podrá observar si los modelos son capaces de generar oraciones coherentes. curiosamente aunque la perplejidad más baja la tiene el modelo unigrama, las oraciones generadas por el modelo trigramas son las que más sentido tienen.

In [68]:
sentence_unigram = generate_sentence(
    model=model_20n_uni,
    initial_context=['the'],
    stop_tokens=['</s>'],
    max_length=10,
    include_context=True
)
print(' '.join(sentence_unigram))

the or NUM satellites 's do the lumping , j article


In [72]:
sentence_unigram = generate_sentence(
    model=model_20n_uni,
    initial_context=['we'],
    stop_tokens=['</s>'],
    max_length=10,
    include_context=True
)
print(' '.join(sentence_unigram))

we . platforms those rick , -- the </s>


In [69]:

sentence_bigram = generate_sentence(
    model=model_bac_bi,
    initial_context=['the'],
    stop_tokens=['</s>'],
    max_length=10,
    include_context=True
)
print(' '.join(sentence_bigram))

the same time . </s>


In [71]:
sentence_bigram = generate_sentence(
    model=model_bac_bi,
    initial_context=['we'],
    stop_tokens=['</s>'],
    max_length=10,
    include_context=True
)
print(' '.join(sentence_bigram))

we were n't know what i 'm not to the same


In [79]:
sentence_bigram = generate_sentence(
    model=model_bac_bi,
    initial_context=['do'],
    stop_tokens=['</s>'],
    max_length=15,
    include_context=True
)
print(' '.join(sentence_bigram))

do n't know what i 'm not to the same time . </s>


In [80]:

sentence = generate_sentence(
    model=model_bac_tri,
    initial_context=['how','do'],
    stop_tokens=['</s>'],
    max_length=15,
    include_context=True
)
print(' '.join(sentence))

how do you think you 're not going to be a good thing . </s>


In [81]:

sentence = generate_sentence(
    model=model_bac_tri,
    initial_context=['<s>','this'],
    stop_tokens=['</s>'],
    max_length=15,
    include_context=True
)
print(' '.join(sentence))

<s> this is the best of all the time . </s>


In [83]:

sentence = generate_sentence(
    model=model_bac_tri,
    initial_context=['the','problem'],
    stop_tokens=['</s>'],
    max_length=15,
    include_context=True
)
print(' '.join(sentence))

the problem is that i have to go to the point of view . </s>
